In [ ]:
# TODO: import stuff
from __future__ import annotations
import sys
import random
import os
import torch
import librosa
import numpy as np
import matplotlib.pyplot as plt
import yaml
import urllib.request
from torch.utils.data import DataLoader, Dataset
import warnings, tqdm

from DL_Project.notebooks.instrument_classifier import visualize_audio

warnings.filterwarnings("ignore", category=tqdm.TqdmWarning)
sys.modules['tqdm.notebook'] = tqdm
sys.modules['tqdm.autonotebook'] = tqdm
from tqdm import tqdm  # now `tqdm(...)` is always the console bar

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Clone the repository
    !git clone https://github.com/ofekdd/DL_Project.git
    %cd DL_Project

    # Install dependencies
    !pip install -r requirements.txt



In [ ]:
from configs import 9cnn.yaml
os.makedirs('configs', exist_ok=True)
cfg = yaml.safe_load(9cnn_config)

print("9cnn configuration:")
for key, value in cfg.items():
    print(f"  {key}: {value}")

In [ ]:
!python /data/download_irmas.py

# TODO: mount, collab or local, GPU

In [ ]:
# Create datasets and dataloaders
train_ds = NpyDataset("data/processed/train")
val_ds = NpyDataset("data/processed/val")

print(f"Training dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=cfg['batch_size'], shuffle=True, num_workers=cfg['num_workers'], collate_fn=pad_collate)
val_loader = DataLoader(val_ds, batch_size=cfg['batch_size'], shuffle=False, num_workers=cfg['num_workers'], collate_fn=pad_collate)


In [ ]:
irmas_root = find_irmas_root()

if irmas_root is None:
    print("❌  No IRMAS audio found. Did you run download_irmas.py yet?")
else:
    print(f"Using IRMAS root: {irmas_root}")
    preprocess_data(irmas_root, "data/processed", cfg)
# TODO: import data normalize, split train/val/test. preprocess: STFT, normalize and reshape

In [ ]:
import models/9cnn_baseline.py

print("9 CNN Baseline Architecture:")
print(MultiSTFTCNN())

# TODO: define model

In [ ]:
model.train
# TODO: train on model, through STFT

In [ ]:
model.test
# TODO: define datapath for test/use: from stft to cnn

In [2]:

visualize_audio(...)
# TODO: test and show results.